<a href="https://colab.research.google.com/github/imdeepmind/UdacitySelfDrivingCar/blob/master/Base_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
# Getting the into colab
!git clone https://github.com/imdeepmind/UdacitySelfDrivingCar.git
!mv UdacitySelfDrivingCar/data ./data
!rm -r UdacitySelfDrivingCar

Cloning into 'UdacitySelfDrivingCar'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 5218 (delta 7), reused 12 (delta 5), pack-reused 5200
Receiving objects: 100% (5218/5218), 66.02 MiB | 10.30 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [145]:
# Importing dependencies
import pandas as pd
import numpy as np
import cv2

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten

In [139]:
class ImageDataGenerator:
  def __init__(self, training_images, validation_images, batch_size=32):
    self.__training_counter = 0
    self.__validation_counter  = 0

    self.__training_images = training_images
    self.__validation_images = validation_images

    self.__batch_size = batch_size

  def __read_images(self, path):
    return cv2.imread(path, 1)
  
  def __image_augmenration(self, image):
    # TODO: Need to do some augmentation
    return image
  
  def __stack_images(self, left, center, right):
    return np.hstack([left, center, right])

  def training_generator(self):
    for i in range(0, len(self.__training_images), self.__batch_size):
      batch_data = self.__training_images[self.__training_counter:self.__training_counter+self.__batch_size]

      images = []
      steers = []

      for element in batch_data:
        center_image = self.__image_augmenration(self.__read_images(element[0]))
        left_image = self.__image_augmenration(self.__read_images(element[1]))
        right_image = self.__image_augmenration(self.__read_images(element[2]))

        image = self.__stack_images(left_image, center_image, right_image)

        images.append(image)
        steers.append(element[3])

      self.__training_counter += self.__batch_size

      yield np.array(images), np.array(steers)
  
  def validation_generator(self):
    for i in range(0, len(self.__validation_images), self.__batch_size):
      batch_data = self.__validation_images[self.__validation_counter:self.__validation_counter+self.__batch_size]

      images = []
      steers = []

      for element in batch_data:
        center_image = self.__read_images(element[0])
        left_image = self.__read_images(element[1])
        right_image = self.__read_images(element[2])

        image = self.__stack_images(left_image, center_image, right_image)

        images.append(image)
        steers.append(element[3])

      self.__validation_counter += self.__batch_size

      yield np.array(images), np.array(steers)

In [146]:
# The Model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = 3, padding = 'Same', activation ='relu', input_shape = (160,960,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 960, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 480, 64)       0         
_________________________________________________________________
dropout (Dropout)            (None, 80, 480, 64)       0         
_________________________________________________________________
flatten (Flatten)            (None, 2457600)           0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2457601   
Total params: 2,459,393
Trainable params: 2,459,393
Non-trainable params: 0
_________________________________________________________________


In [147]:
data = pd.read_csv('./data/driver_log_cleaned.csv')

train_data, test_data = train_test_split(data.values, test_size=0.1)

In [148]:
gen = ImageDataGenerator(train_data, test_data)

training_generator = gen.training_generator()
validation_generator = gen.validation_generator()

In [149]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

In [150]:
model.fit(training_generator, steps_per_epoch=100, validation_data=validation_generator, validation_steps=10, epochs=1)

  3/100 [..............................] - ETA: 5:21 - loss: 239969168.0000 - mae: 10676.6094

KeyboardInterrupt: ignored